## Demo Notebook for training a DGAT model.

Here, we trained a DGAT model using only one sample, which is Tonsil dataset. You can replace this with your own dataset. The dataset should be in the format of AnnData object with RNA and protein data. After that we predicted the protein expression from the RNA data of Lymph Node in the **Demo_Predict_2.ipynb**.

Please replace the **sample_list**, **model_save_dir**, **pyg_save_dir** and **dataset_save_dir** with your actual paths

In [1]:
import numpy as np

import scanpy as sc
import torch
import warnings
warnings.filterwarnings('ignore')

from utils.Preprocessing import qc_control_cytassist, normalize, clean_protein_names, preprocess_train_list
from Model.Train_and_Predict import train

import random
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

sample_list = ['Tonsil']

dataset_save_dir = './DGAT_training_datasets'

pyg_save_dir = './pyg_data' # Any dir that you want. Building a graph might take 3 mins or more (depends on the scale of the dataset).This directory is used to save the graph data in PyG format, which can be reused for training.

model_save_dir = './DGAT_pretrained_models'# Directory to save the trained DGAT model.


os.makedirs(model_save_dir, exist_ok=True)

### Loading dataset
You can replace this with your own dataset. The dataset should be in the format of AnnData object with RNA and protein data.

In [2]:
adata_list = []
pdata_list = []
for sample in sample_list:
    print(sample)
    adata = sc.read_h5ad(f'{dataset_save_dir}/{sample}_RNA.h5ad')
    pdata = sc.read_h5ad(f'{dataset_save_dir}/{sample}_ADT.h5ad')
    adata_list.append(adata)
    pdata_list.append(pdata)


Tonsil


### Preprocessing
In this step, we will perform quality control, normalization, and finding the common genes and proteins across the datasets.


In [3]:
# Performs quality control and normalization on the training datasets
common_gene, common_protein = preprocess_train_list(adata_list, pdata_list)

Common genes before QC: 18085
Common proteins before QC: 35
Num of encoding genes: 30
Common genes after QC: 17434
Common proteins after QC: 31
Common gene names saved to common_gene_17434.txt
Common protein names saved to common_protein_31.txt


In [4]:
adata_list = [adata[:,common_gene] for adata in adata_list]
pdata_list = [pdata[:,common_protein] for pdata in pdata_list]

### Training



In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Trains the DGAT model using the preprocessed training datasets. It takes in the list of processed ST datasets and protein datasets, and the directory to save the processed graph data (for more efficient reproduction).
model_components = train(adata_list, pdata_list, pyg_save_dir)

model_save_path = f"{model_save_dir}/{len(common_gene)}_gene_{len(common_protein)}_protein"
print(model_save_path)
os.makedirs(model_save_path, exist_ok=True)
torch.save(model_components['mRNA_encoder'].state_dict(), model_save_path+f'/encoder_mRNA.pth')
torch.save(model_components['mRNA_decoder'].state_dict(), model_save_path+f'/decoder_mRNA.pth')
torch.save(model_components['protein_encoder'].state_dict(), model_save_path+f'/encoder_protein.pth')
torch.save(model_components['protein_decoder'].state_dict(), model_save_path+f'/decoder_protein.pth')



Common genes: 17434
Common proteins: 31
Creating or loading dataset
Creating and saving preprocessed data for sample 'CytAssist_FFPE_Protein_Expression_Human_Tonsil'
Dataset ready
Using single device for both models


Epoch [1/100] Total Loss: 12.9259 | mRNA Recon: 1.3340 | Protein Recon: 0.9171 | Alignment: 1.2841 | Protein Pred: 0.9071 | mRNA Pred: 1.3334
Epoch 1: EB evidence = -2.4350


Epoch [2/100] Total Loss: 10.9490 | mRNA Recon: 1.3014 | Protein Recon: 0.5432 | Alignment: 1.0218 | Protein Pred: 0.5248 | mRNA Pred: 1.3022
Epoch 2: EB evidence = -1.4304


Epoch [3/100] Total Loss: 9.9475 | mRNA Recon: 1.2659 | Protein Recon: 0.4257 | Alignment: 0.7465 | Protein Pred: 0.3922 | mRNA Pred: 1.2692
Epoch 3: EB evidence = -0.2269


Epoch [4/100] Total Loss: 9.3565 | mRNA Recon: 1.2305 | Protein Recon: 0.3731 | Alignment: 0.6035 | Protein Pred: 0.3312 | mRNA Pred: 1.2340
Epoch 4: EB evidence = 4.7506


...

Epoch [47/100] Total Loss: 6.8642 | mRNA Recon: 0.9963 | Protein Recon: 0.1676 | Alignment: 0.2188 | Protein Pred: 0.1660 | mRNA Pred: 0.9981
Epoch 47: EB evidence = 1.0051


Epoch [48/100] Total Loss: 6.8558 | mRNA Recon: 0.9959 | Protein Recon: 0.1672 | Alignment: 0.2173 | Protein Pred: 0.1647 | mRNA Pred: 0.9978
Epoch 48: EB evidence = 1.0267


Epoch [49/100] Total Loss: 6.8481 | mRNA Recon: 0.9954 | Protein Recon: 0.1660 | Alignment: 0.2158 | Protein Pred: 0.1640 | mRNA Pred: 0.9973
Epoch 49: EB evidence = 0.9778


Epoch [50/100] Total Loss: 6.8377 | mRNA Recon: 0.9949 | Protein Recon: 0.1652 | Alignment: 0.2141 | Protein Pred: 0.1624 | mRNA Pred: 0.9969
Epoch 50: Learning rate reduced.
Epoch 50: EB evidence = 1.0086


Epoch [51/100] Total Loss: 6.8277 | mRNA Recon: 0.9943 | Protein Recon: 0.1639 | Alignment: 0.2123 | Protein Pred: 0.1611 | mRNA Pred: 0.9965
Epoch 51: EB evidence = 0.9553
--> EB early stopping at epoch 51 (mean_evidence=0.9897, threshold=0.96)
./DGAT_models/17434_gene_31_protein
